In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/qwen2/transformers/1.5b-instruct/1/config.json
/kaggle/input/qwen2/transformers/1.5b-instruct/1/merges.txt
/kaggle/input/qwen2/transformers/1.5b-instruct/1/LICENSE
/kaggle/input/qwen2/transformers/1.5b-instruct/1/README.md
/kaggle/input/qwen2/transformers/1.5b-instruct/1/tokenizer.json
/kaggle/input/qwen2/transformers/1.5b-instruct/1/vocab.json
/kaggle/input/qwen2/transformers/1.5b-instruct/1/tokenizer_config.json
/kaggle/input/qwen2/transformers/1.5b-instruct/1/model.safetensors
/kaggle/input/qwen2/transformers/1.5b-instruct/1/generation_config.json
/kaggle/input/qwen2/transformers/7b-instruct/1/model.safetensors.index.json
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00003-of-00004.safetensors
/kaggle/input/qwen2/transformers/7b-instruct/1/config.json
/kaggle/input/qwen2/transformers/7b-instruct/1/merges.txt
/kaggle/input/qwen2/transformers/7b-instruct/1/LICENSE
/kaggle/input/qwen2/transformers/7b-instruct/1/model-00001-of-00004.safetensors
/kaggle/input/qwen2/t

In [ ]:
import torch
print("GPU0 free:", torch.cuda.mem_get_info(0)[0] / 1024**3, "GB")
print("GPU1 free:", torch.cuda.mem_get_info(1)[0] / 1024**3, "GB")


In [ ]:
#!pip install --quiet transformers sentence-transformers tqdm accelerate


In [ ]:
#!pip install --quiet chromadb gradio

In [ ]:
#!pip install protobuf==3.20.3 --quiet --force-reinstall

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import os, json, torch, time
from datasets import load_dataset
import chromadb
from chromadb.utils import embedding_functions
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as gr
from typing import List, Tuple, Dict

# Paths
CHROMA_PATH = "/kaggle/working/chroma_db"
QWEN_PATH = "/kaggle/working/Qwen2-7B-Instruct"  # Change if different
COLLECTION_NAME = "health_rag_collection"

# Models
EMBED_MODEL = "sentence-transformers/all-MiniLM-L6-v2"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Agent settings
TOPK_INITIAL = 5
TOPK_EXPANDED = 10
CONF_THRESHOLD = 0.75


In [ ]:
# Core datasets
ds_health = load_dataset("katielink/healthsearchqa")       # Layman queries
ds_mediqa = load_dataset("bigbio/mediqa_qa")               # Patient-oriented answers
ds_medquad = load_dataset("lavita/MedQuAD")                # Medical grounding

print("Datasets Loaded:")
print("HealthSearchQA:", len(ds_health['train']))
print("MediQA-QA:", len(ds_mediqa['train']))
print("MedQuAD:", len(ds_medquad['train']))


In [ ]:
client = chromadb.PersistentClient(path=CHROMA_PATH)
embed_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)
collection = client.get_or_create_collection(name=COLLECTION_NAME, embedding_function=embed_fn)


In [ ]:
if collection.count() == 0:
    docs, metas, ids = [], [], []
    for row in ds_medquad['train'][:3000]:  # limit for Kaggle
        text = row.get('answer', '') or row.get('description','')
        docs.append(text)
        metas.append({"topic": row.get('disease', ''), "source": "MedQuAD"})
        ids.append(str(len(ids)))
    collection.add(documents=docs, metadatas=metas, ids=ids)
    print("Ingested to Chroma:", collection.count(), "documents")
else:
    print("Chroma index already exists:", collection.count())


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(QWEN_PATH)
model = AutoModelForCausalLM.from_pretrained(QWEN_PATH, torch_dtype="auto", device_map="auto")
model.eval()

def chat(system, user, max_tokens=512):
    msgs=[{"role":"system","content":system},{"role":"user","content":user}]
    text=tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=True)
    inputs=tokenizer(text, return_tensors="pt").to(model.device)
    with torch.no_grad():
        out=model.generate(**inputs, max_new_tokens=max_tokens, temperature=0.0)
    return tokenizer.decode(out[0, inputs['input_ids'].shape[1]:], skip_special_tokens=True).strip()


In [ ]:
def retrieve(query, k=TOPK_INITIAL):
    result = collection.query(query_texts=[query], n_results=k)
    return list(zip(result['documents'][0], result['metadatas'][0]))

def format_context(docs):
    return "\n\n".join([f"[Doc {i+1}] {d[:400]}" for i,(d,_) in enumerate(docs)])

# Layman reformulation
def reformulate_query(query):
    sys = "You convert patient queries into medically precise search terms."
    return chat(sys, f"Original: {query}\nReformulate: ")

# Clarify question
def generate_clarify(query):
    sys="If query is unclear, ask ONE follow-up question."
    return chat(sys, f"Patient query: {query}\nAsk clarification only if needed:")

# Risk module
def risk_awareness(response):
    sys = "Provide risk awareness (non-advice) based on this answer"
    return chat(sys, f"Based on: {response}\nGenerate education-only risk awareness:")

# Doctor discussion prompt
def doctor_questions(response):
    sys = "Give 1-2 questions patient should ask doctor (no advice)"
    return chat(sys, f"Response: {response}\nSuggest questions:")


In [ ]:
def agentic_rag(query, mode="patient"):
    clarify = generate_clarify(query)
    if "?" in clarify:
        query = clarify

    med_query = reformulate_query(query)
    docs = retrieve(med_query)

    sys = "You answer using only context."
    usr = f"Q: {query}\nContext:\n{format_context(docs)}"
    answer = chat(sys, usr)

    eval_sys = "Evaluate response in JSON: {'score':0-1,'needs_retry':bool}"
    eval_res = chat(eval_sys, f"Q:{query}\nAnswer:{answer}")
    try:
        ev = json.loads(eval_res)
    except:
        ev = {"score":0.0,"needs_retry":True}

    if ev['score'] < CONF_THRESHOLD:
        docs = retrieve(med_query, TOPK_EXPANDED)
        answer = chat(sys, f"Q:{query}\nContext:\n{format_context(docs)}")

    if mode=="patient":
        answer = chat("Simplify & support emotionally", f"{answer}\nPatient version:")
        answer += "\n\n⚠️ " + risk_awareness(answer)
        answer += "\n\n🩺 Questions to ask your doctor:\n- " + doctor_questions(answer)

    return answer, ev


In [ ]:
# 1. Test queries
test_queries = [
    "I have high blood pressure and I’m going for surgery. Should I worry?",
    "I have diabetes and I'm scheduled for cataract surgery. Is it risky?",
    "I have asthma but I often ignore inhalers. Can it affect anesthesia?",
    "I take multiple heart medications. Should I inform my doctor before treatment?",
    "I often skip sugar check-ups. Will it slow down recovery after surgery?"
]

# 2. Evaluate system
from time import time

def evaluate_system(queries, mode="patient"):
    results = []
    for q in queries:
        start = time()
        ans, ev = agentic_rag(q, mode)
        elapsed = time() - start
        
        results.append({
            "query": q,
            "confidence_score": ev.get("score", None),
            "needed_retry": ev.get("needs_retry", None),
            "response_time_sec": round(elapsed, 2),
            "response_preview": ans[:150] + "..."  # avoid large outputs
        })
    return results

results = evaluate_system(test_queries)
import pandas as pd
df_results = pd.DataFrame(results)

# 3. Summary metrics
summary = {
    "avg_confidence": round(df_results["confidence_score"].mean(), 3),
    "retry_rate_%": round((df_results["needed_retry"].sum() / len(df_results)) * 100, 2),
    "avg_response_time_sec": round(df_results["response_time_sec"].mean(), 2)
}

df_results, summary


In [ ]:
def ui_handler(query, mode):
    try:
        ans, evaluation = agentic_rag(query, mode)
        return ans, json.dumps(evaluation, indent=2)
    except Exception as e:
        return f"Error: {e}", "N/A"

with gr.Blocks() as ui:
    gr.Markdown("# 🤖 Agentic RAG Healthcare Assistant\n(MVP Prototype)")
    inp = gr.Textbox(label="Enter your question")
    mode = gr.Radio(["patient", "professional"], value="patient", label="Response Mode")
    btn = gr.Button("Generate")
    out = gr.Markdown()
    ev = gr.JSON()

    btn.click(ui_handler, inputs=[inp, mode], outputs=[out, ev])

ui.launch(share=False)
